In [8]:
import os

fileList=os.listdir('./data/html/')

In [9]:
from bs4 import BeautifulSoup
soups=[]
for file in fileList:
    with open(os.path.join('./data/html/',file)) as f:
        
        soup=BeautifulSoup(f.read())
        soups.append(soup)

In [10]:
print((soups[0].find_all('p')[7]).text)

On 11 February, two evacuees from Wuhan tested positive after an earlier test returned negative results.[18] Another three cases were confirmed over the next two days, bringing the total count to 31.[19]



In [13]:
import spacy
nlp = spacy.load("en_core_web_sm")

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [ ]:
docList=[]
for p in soups[0].find_all('p'):
    docList.append(nlp(p.text))

In [23]:
doc=docList[3]

In [24]:
doc

As of 18 March 2020, South Korea has about 8,400 cases and 84 deaths,[9] with over 200,000 people having been tested,[10][11] a case fatality rate of 1.0%, which is lower than the WHO's global case fatality rate of 3.4%.[12]

In [15]:
for doc in docList:
    print(doc)



The first confirmed case of the pandemic of coronavirus disease 2019 (COVID-19) in South Korea was announced on 20 January 2020.[1] On 19 February, the number of confirmed cases increased by 20 and on 20 February by 58[2] or 70,[3] giving a total of 346 confirmed cases on 21 February 2020, according to the Centers for Disease Control and Prevention Korea (KCDC), with the sudden jump mostly attributed to "Patient 31" who participated in a gathering at a Shincheonji Church of Jesus the Temple of the Tabernacle of the Testimony church in Daegu.[3][4]

Amidst the fears of further contamination, mass gatherings in the affected cities were cancelled and a few hundred soldiers in Daegu are in isolation.[5][6] As of 4 Feb, South Korea denies entry to foreigners traveling from Hubei Province to help prevent the spread of COVID-19.[7][8]

As of 18 March 2020, South Korea has about 8,400 cases and 84 deaths,[9] with over 200,000 people having been tested,[10][11] a case fatality rate of 1.0%, w

Die Entity-Recognition von Spacy funktioniert schlecht:

In [16]:
for token in doc:
    if token.ent_type_ != None:
        print(token,token.ent_type_)

  
Media 
related 
to 
2019–20 
COVID-19 
pandemic 
in 
South GPE
Korea GPE
at 
Wikimedia ORG
Commons ORG

 


In [9]:
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab)

In [33]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)

pattern = [ {"IS_DIGIT":True},{"TEXT": "January"}]
pattern2 = [ {"IS_DIGIT":True},{"TEXT": "February"}]
pattern3 = [ {"IS_DIGIT":True},{"TEXT": "March"}]

matcher.add("JAN", None, pattern)
matcher.add("FEB", None, pattern2)
matcher.add("MARCH", None, pattern3)

In [28]:
matches = matcher(doc)
print("Matches:", [doc[start:end].text for match_id, start, end in matches])

Matches: ['18 March']


In [34]:
for doc in docList:
    matches = matcher(doc)
    print("Matches:", [doc[start:end].text for match_id, start, end in matches])

Matches: []
Matches: ['20 January', '19 February', '20 February', '21 February']
Matches: []
Matches: ['18 March']
Matches: []
Matches: ['18 February', '27 February', '10 March']
Matches: ['20 January']
Matches: ['26 January']
Matches: ['27 January', '20 January', '21 January']
Matches: ['1 February']
Matches: ['30 January', '24 January']
Matches: ['31 January', '23 January', '26 January']
Matches: ['1 February', '2 February']
Matches: ['5 February']
Matches: []
Matches: ['18 February']
Matches: ['19 February', '20 February']
Matches: ['20 February', '21 February']
Matches: ['26 February']
Matches: ['22 February', '23 February', '24 February']
Matches: ['24 February']
Matches: ['28 February', '2 March', '8 March']
Matches: ['17 March']
Matches: []
Matches: []
Matches: []
Matches: []
Matches: []
Matches: []
Matches: []
Matches: []
Matches: []
Matches: []
Matches: []
Matches: ['18 March', '23 February', '15 March']
Matches: ['25 February']
Matches: []
Matches: ['18 February', '16 March',